In [1]:
# coding: UTF-8
import os
import torch
import numpy as np
import pickle as pkl
from tqdm import tqdm
import time
from datetime import timedelta
import time
import torch
import numpy as np
from train_eval import train, init_network
from importlib import import_module
import argparse
dataset = 'Newtrain'  # 数据集

# 搜狗新闻:embedding_SougouNews.npz, 腾讯:embedding_Tencent.npz, 随机初始化:random
embedding = 'embedding_Newtrain_300.npz'

model_name = 'TextRCNN'  # 'TextRCNN'  # TextCNN, TextRNN, FastText, TextRCNN, TextRNN_Att, DPCNN, Transformer


x = import_module('models.' + model_name)
config = x.Config(dataset, embedding)
config.pad_size = 64

MAX_VOCAB_SIZE = 20000  # 词表长度限制
UNK, PAD = '<UNK>', '<PAD>'  # 未知字，padding符号
ues_word = True
from utils import get_time_dif, build_vocab, build_dataset2, build_iterator2


vocab, train, test = build_dataset2(config, ues_word)

Vocab size: 12678


8000it [00:00, 67541.40it/s]
2653it [00:00, 68204.87it/s]


In [2]:
import torch.nn.functional as F


train_iter = build_iterator2(train, config)
test_iter = build_iterator2(test, config)
# train_iter.__next__()
config.n_vocab = len(vocab)
model = x.Model(config).to(config.device)
if model_name != 'Transformer':
    init_network(model)
print(model.parameters)

<bound method Module.parameters of Model(
  (embedding): Embedding(12678, 300)
  (lstm): LSTM(300, 256, batch_first=True, dropout=1.0, bidirectional=True)
  (maxpool): MaxPool1d(kernel_size=64, stride=64, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=812, out_features=10, bias=True)
)>


/opt/conda/lib/python3.7/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=1.0 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
import numpy as np
import torch
import json
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
import time
from utils import get_time_dif
from tensorboardX import SummaryWriter
from collections import defaultdict
def train_zhuti(config, model, train_iter, dev_iter):
    start_time = time.time()
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    loss_fn = nn.BCEWithLogitsLoss()
    # 学习率指数衰减，每次epoch：学习率 = gamma * 学习率
    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    total_batch = 0  # 记录进行到多少batch
    dev_best_loss = float('inf')
    alpha = 0.5
    last_improve = 0  # 记录上次验证集loss下降的batch数
    flag = False  # 记录是否很久没有效果提升
    train_dict = defaultdict(list)
    test_dict = defaultdict(list)
    
    writer = SummaryWriter(log_dir=config.log_path + '/' + time.strftime('%m-%d_%H.%M', time.localtime()))
    for epoch in range(config.num_epochs):
        print('Epoch [{}/{}]'.format(epoch + 1, config.num_epochs))
        # scheduler.step() # 学习率衰减
        for i, (trains, qinggan_label, zhuti_label) in enumerate(train_iter):
            outputs = model(trains)
            model.zero_grad()
            # loss = F.cross_entropy(outputs, qinggan_label)
            zhuti_label = zhuti_label.float()
            # print(outputs, zhuti_label)
            loss = loss_fn(outputs, zhuti_label)
            loss.backward()
            optimizer.step()
            if total_batch % 1 == 0:
                # 每多少轮输出在训练集和验证集上的效果
                predic = (torch.sigmoid(outputs) > alpha).float()
                # true = qinggan_label.data.cpu()
                # predic = torch.max(outputs.data, 1)[1].cpu()
                             
                train_acc = metrics.accuracy_score(predic.view(-1), zhuti_label.view(-1))
                train_precision = metrics.precision_score(predic.view(-1), zhuti_label.view(-1), average='macro', zero_division=0)
                train_recall = metrics.recall_score(predic.view(-1), zhuti_label.view(-1), average='macro', zero_division=0)
                   
                dev_acc, dev_loss, dev_p, dev_r = evaluate_zhuti(config, model, dev_iter)
                                
                if dev_loss < dev_best_loss:
                    dev_best_loss = dev_loss
                    torch.save(model.state_dict(), config.save_path)
                    improve = '*improve'
                    last_improve = total_batch
                else:
                    improve = ''
                time_dif = get_time_dif(start_time)
                msg = 'Iter: {0:>6},  Train Loss: {1:>5.2},  Train Acc: {2:>6.2%},  Val Loss: {3:>5.2},  Val Acc: {4:>6.2%},  Time: {5} {6}'
                print(msg.format(total_batch, loss.item(), train_acc, dev_loss, dev_acc, time_dif, improve))
                msg2 = 'Iter: {0:>6},  Train precision: {1:>5.2%},  Train recall: {2:>6.2%},  Val precision: {3:>5.2%},  Val recall: {4:>6.2%},  Time: {5} {6}'
                print(msg2.format(total_batch, train_precision, train_recall, dev_p, dev_r, time_dif, improve))
                writer.add_scalar("loss/train", loss.item(), total_batch)
                writer.add_scalar("loss/dev", dev_loss, total_batch)
                writer.add_scalar("acc/train", train_acc, total_batch)
                writer.add_scalar("precision/train", train_precision, total_batch)
                writer.add_scalar("recall/train", train_recall, total_batch)
                writer.add_scalar("acc/dev", dev_acc, total_batch)
                writer.add_scalar("precision/dev", dev_p, total_batch)
                writer.add_scalar("recall/dev", dev_r, total_batch)
                train_dict['train_loss'].append(loss.item())   
                train_dict['train_acc'].append(train_acc)
                train_dict['train_precision'].append(train_precision)
                train_dict['train_recall'].append(train_recall)
                test_dict['dev_acc'].append(dev_acc)
                test_dict['dev_loss'].append(dev_loss.item())
                test_dict['dev_precision'].append(dev_p)
                test_dict['dev_recall'].append(dev_r)
                with open(config.log_path + '/train_dict.json', 'w') as json_file:
                    json.dump(train_dict, json_file, indent=4)  # indent参数使输出更加易读
                with open(config.log_path + '/test_dict.json', 'w') as json_file:
                    json.dump(test_dict, json_file, indent=4)  # indent参数使输出更加易读
                model.train()
            total_batch += 1
            if total_batch - last_improve > config.require_improvement:
                # 验证集loss超过1000batch没下降，结束训练
                print("No optimization for a long time, auto-stopping...")
                flag = True
                break
        if flag:
            break
    writer.close()
def evaluate_zhuti(config, model, data_iter, test=False):
    model.eval()
    loss_total = 0
    alpha = 0.5
    predict_all = np.array([], dtype=int)
    labels_all = np.array([], dtype=int)
    loss_fn = nn.BCEWithLogitsLoss()
    with torch.no_grad():
        for texts, qinggan_label, zhuti_label in data_iter:
            outputs = model(texts)
            zhuti_label = zhuti_label.float()
            loss = loss_fn(outputs, zhuti_label)
            loss_total += loss
            labels = zhuti_label.data.cpu().numpy()
            # predic = torch.max(outputs.data, 1)[1].cpu().numpy()
            predic = (torch.sigmoid(outputs) > alpha).float().cpu().numpy()
            labels_all = np.append(labels_all, labels)
            predict_all = np.append(predict_all, predic)

    acc = metrics.accuracy_score(labels_all.ravel(), predict_all.ravel())
    precision = metrics.precision_score(labels_all.ravel(), predict_all.ravel(), average='macro', zero_division=0)
    recall = metrics.recall_score(labels_all.ravel(), predict_all.ravel(), average='macro', zero_division=0)
    if test:
        report = metrics.classification_report(labels_all, predict_all, target_names=config.class_list, digits=4)
        confusion = metrics.confusion_matrix(labels_all, predict_all)
        return acc, loss_total / len(data_iter), report, confusion
    return acc, loss_total / len(data_iter), precision, recall
train_zhuti(config, model, train_iter, test_iter)

Epoch [1/10]
Iter:      0,  Train Loss:  0.69,  Train Acc: 45.62%,  Val Loss:  0.51,  Val Acc: 86.02%,  Time: 0:05:47 *improve
Iter:      0,  Train precision: 52.68%,  Train recall: 51.00%,  Val precision: 51.66%,  Val recall: 50.63%,  Time: 0:05:47 *improve
Iter:      1,  Train Loss:  0.53,  Train Acc: 83.67%,  Val Loss:   0.4,  Val Acc: 88.76%,  Time: 0:08:43 *improve
Iter:      1,  Train precision: 51.34%,  Train recall: 51.70%,  Val precision: 44.40%,  Val recall: 49.98%,  Time: 0:08:43 *improve
Iter:      2,  Train Loss:   0.4,  Train Acc: 89.22%,  Val Loss:  0.35,  Val Acc: 88.80%,  Time: 0:10:13 *improve
Iter:      2,  Train precision: 49.87%,  Train recall: 44.71%,  Val precision: 44.40%,  Val recall: 50.00%,  Time: 0:10:13 *improve


In [2]:
from sklearn import metrics
def APH(y_true, y_pred):
    return metrics.accuracy_score(y_true, y_pred), \
           metrics.precision_score(y_true, y_pred, average='macro'), metrics.recall_score(y_true, y_pred, average='macro')
APH([0,1,1,1,0, 0, 0, 1],[0,0,0,0,1, 1, 0, 0])

(0.25, 0.16666666666666666, 0.25)

In [6]:
z = train_iter.__next__()[2]

In [12]:
z = z.float()

In [13]:
import torch
import torch.nn as nn

# 设置随机种子以获得可复现的结果
torch.manual_seed(42)

# 假设我们有4个样本，每个样本有5个类别，可能同时属于多个类别
num_samples = 128
num_classes = 10

# 随机生成预测logits（未经过sigmoid激活）
random_logits = torch.randn(num_samples, num_classes)

# 随机生成真实标签，这里用0或1表示不属于或属于某个类别，对于多标签任务，一个样本可以有多个类别为1
random_labels = torch.bernoulli(torch.rand(num_samples, num_classes)).float()

# 初始化 BCEWithLogitsLoss 损失函数
loss_fn = nn.BCEWithLogitsLoss()

# 计算损失
loss = loss_fn(random_logits, z)

print("Random Logits:\n", random_logits)
# print("Random Labels:\n", random_labels)
print("Calculated Loss:", loss.item())

Random Logits:
 tensor([[ 1.9269,  1.4873,  0.9007,  ..., -1.6047, -0.7521,  1.6487],
        [-0.3925, -1.4036, -0.7279,  ..., -0.1596, -0.4974,  0.4396],
        [-0.7581,  1.0783,  0.8008,  ...,  1.3347, -0.2316,  0.0418],
        ...,
        [-0.7691,  0.0793, -0.7585,  ..., -1.1371, -0.7582,  0.0423],
        [-0.6901, -0.5622,  0.8253,  ...,  1.0238,  0.9575,  0.0191],
        [-1.0700, -0.7519,  2.4401,  ..., -0.1173, -0.6305, -1.2655]])
Calculated Loss: 0.8070029020309448


In [39]:
def Predict(outputs, alpha=0.5):
    predic = (torch.sigmoid(outputs) > alpha).float()

    return predic.cpu()
p = Predict(random_logits)

In [40]:
APH(p.view(-1), random_labels.view(-1)), p, random_labels

((0.6, 0.5858585858585859, 0.5934065934065934),
 tensor([[1., 0., 0., 1., 0.],
         [1., 0., 0., 1., 1.],
         [0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 1.]]),
 tensor([[1., 0., 0., 0., 1.],
         [1., 1., 0., 0., 1.],
         [0., 1., 0., 0., 1.],
         [1., 1., 0., 0., 0.]]))